# Keypoint Moseq

## Project Setup
This notebook was written using the keypoint_moseq example notebooks for Jupyter and Colab as a guide. 
https://github.com/dattalab/keypoint-moseq/blob/main/docs/source/modeling.ipynb

This notebook is setup to analyze keypoint data from DeepLabCuts SuperAnimal-TopViewMouse model. Data produced using other models will need a different setup.

**Important Note About Paths**:  
On WSL, do NOT use the backslash "\\" in your paths. They will not function correctly. Use the ordinary "/".
Before your path, use the letter "r". 
Here is an example of a path that has run correctly using this notebook from WSL: 
> "r'/mnt/c/Users/hray0/OneDrive/Desktop/231206_10min/l_only/l_cropped/videos/'"

### Import keypoint_moseq
If you are on windows 11, this must be done is Windows Subsystem for Linux.

In [ ]:
import keypoint_moseq as kpms

In [ ]:
project_dir = r'/mnt/c/Users/hray0/OneDrive/Desktop/test_dir/lonly_test'
config = lambda: kpms.load_config(project_dir)

If you need to find the config.yaml file for the SuperAnimal-TopViewMouse model:
On Windows the path to this file should look something like "C:\Users\your_user\anaconda3\envs\DEEPLABCUT\Lib\site-packages\deeplabcut\pose_estimation_tensorflow\superanimal_configs\supertopview.yaml"

### Manual Set Up
For more information about the bodyparts used in the SuperAnimal-TopviewMouse model see "SuperAnimal pretrained pose estimation models for behavioral analysis." https://arxiv.org/abs/2203.07436

In [ ]:
# The code below sets up the project manually for data obtained using the SuperAnimal-TopViewMouse model
bodyparts=[
    'nose', 'left_ear', 'right_ear', 'left_ear_tip', 'right_ear_tip',
    'left_eye', 'right_eye', 'neck', 'mid_back', 'mouse_center', 'mid_backend',
    'mid_backend2', 'mid_backend3', 'tail_base', 'tail1', 'tail2', 'tail3',
    'tail4', 'tail5', 'left_shoulder', 'left_midside', 'left_hip', 'right_shoulder',
    'right_midside', 'right_hip', 'tail_end', 'head_midpoint',] 

skeleton=[
    ['tail_base', 'mid_backend3'],
    ['right_hip', 'mid_backend3'],
    ['left_hip', 'mid_backend3'],
    ['mid_backend3', 'mid_backend2'],
    ['mid_backend2', 'mid_backend'],
    ['mid_backend', 'mouse_center'],
    ['left_midside', 'mouse_center'],
    ['right_midside', 'mouse_center'],
    ['mouse_center', 'mid_back'],
    ['left_shoulder', 'mid_back'],
    ['right_shoulder', 'mid_back'],
    ['mid_back', 'neck'],
    ['neck', 'head_midpoint'],
    ['head_midpoint', 'left_ear'],
    ['head_midpoint', 'right_ear'],
    ['left_ear', 'left_ear_tip'],
    ['right_ear', 'right_ear_tip'],
    ['head_midpoint', 'left_eye'],
    ['head_midpoint', 'right_eye'],
    ['head_midpoint', 'nose'],]

video_dir= r'mnt/c/Users/hray0/OneDrive/Desktop/231206_10min/l_only/l_cropped/videos/' #directory with videos of each experiment 

kpms.setup_project(
    project_dir,
    video_dir=video_dir,
    bodyparts=bodyparts,
    skeleton=skeleton)

### Edit the Config
This step is critical, because it specifies which bodyparts moseq will use in its analysis. Typically, the keypoint data will include all of the bodyparts available in the DLC topview model. Using fewer keypoints is advisable, and omitting the tail is recommended. I've set up the following cell to use 10 keypoints.

In [ ]:
kpms.update_config(
    project_dir,
    video_dir=r'/mnt/c/Users/hray0/OneDrive/Desktop/231206_10min/l_only/l_cropped/videos/',
    anterior_bodyparts=['nose'],
    posterior_bodyparts=['tail_base'],
    use_bodyparts=[
        'nose', 'left_ear', 'right_ear', 'neck', 'mid_back', 'mouse_center', 'mid_backend',
        'mid_backend2', 'mid_backend3', 'tail_base',])

# use_bodyparts tells moseq which keypoints to use from the DLC data. 
# The keypoint-moseq docs recommend omitting the tail. 

## Working with Data

### Load Data

In [ ]:
# Make sure your keypoint data is in the folder labeled 'videos.'  
# load data from DeepLabCut
keypoint_data_path = r'/mnt/c/Users/hray0/OneDrive/Desktop/231206_10min/l_only/l_cropped/videos/' # can be a file, a directory, or a list of files
coordinates, confidences, bodyparts = kpms.load_keypoints(keypoint_data_path, 'deeplabcut')

# format data for modeling
data, metadata = kpms.format_data(coordinates, confidences, **config())

### Calibration

This step requires jupyter lab. It will not work in jupyter notebook.

The purpose of calibration is to learn the relationship between keypoint error and confidence scores, which is stored in the config as a pair of slope and intercept coefficients. One can also adjust the confidence_threshold parameter at this step, which is used to define outlier keypoints for PCA and model initialization. This step can be skipped for the demo data.

    Run the cell below. A widget should appear with a video frame on the left.

    Annotate each frame with the correct location of the labeled bodypart

        Left click to specify the correct location - an “X” should appear.

        Use the arrow buttons to annotate additional frames.

        Each annotation adds a point to the right-hand scatter plot.

        Continue until the regression line stabilizes.

    At any point, adjust the confidence threshold by clicking on the scatter plot.

    Use the “save” button to update the config and store your annotations to disk.


In [ ]:
kpms.noise_calibration(project_dir, coordinates, confidences, **config())

### Fit PCA

In [ ]:
pca = kpms.fit_pca(**data, **config())
kpms.save_pca(pca, project_dir)

kpms.print_dims_to_explain_variance(pca, 0.9)
kpms.plot_scree(pca, project_dir=project_dir)
kpms.plot_pcs(pca, project_dir=project_dir, **config())

# use the following to load an already fit model
# pca = kpms.load_pca(project_dir)

In [ ]:
kpms.update_config(project_dir, latent_dim=7)

### Model Fitting

#### Setting Kappa

Automatic Kappa Scan

In [ ]:
import numpy as np

kappas = np.logspace(3,7,5)
decrease_kappa_factor = 10
num_ar_iters = 50
num_full_iters = 200

prefix = 'my_kappa_scan'

for kappa in kappas:
    print(f"Fitting model with kappa={kappa}")
    model_name = f'{prefix}-{kappa}'
    model = kpms.init_model(data, pca=pca, **config())

    # stage 1: fit the model with AR only
    model = kpms.update_hypparams(model, kappa=kappa)
    model = kpms.fit_model(
        model,
        data,
        metadata,
        project_dir,
        model_name,
        ar_only=True,
        num_iters=num_ar_iters,
        save_every_n_iters=25
    )[0];

    # stage 2: fit the full model
    model = kpms.update_hypparams(model, kappa=kappa/decrease_kappa_factor)
    kpms.fit_model(
        model,
        data,
        metadata,
        project_dir,
        model_name,
        ar_only=False,
        start_iter=num_ar_iters,
        num_iters=num_full_iters,
        save_every_n_iters=25
    );

kpms.plot_kappa_scan(kappas, project_dir, prefix)

#### Initialization

In [ ]:
# initialize the model
model = kpms.init_model(data, pca=pca, **config())

# optionally modify kappa
model = kpms.update_hypparams(model, kappa=1e6)

#### Fitting an AR-HMM

In [ ]:
num_ar_iters = 50

model, model_name = kpms.fit_model(
    model, data, metadata, project_dir,
    ar_only=True, num_iters=num_ar_iters)

#### Fitting the Full Model

In [ ]:
# load model checkpoint
model, data, metadata, current_iter = kpms.load_checkpoint(
    project_dir, model_name, iteration=num_ar_iters)

# modify kappa to maintain the desired syllable time-scale
model = kpms.update_hypparams(model, kappa=1e4)

# run fitting for an additional 500 iters
model = kpms.fit_model(
    model, data, metadata, project_dir, model_name, ar_only=False, 
    start_iter=current_iter, num_iters=current_iter+500)[0] 

#### Sort Syllables by Frequency

In [ ]:
# modify a saved checkpoint so syllables are ordered by frequency
kpms.reindex_syllables_in_checkpoint(project_dir, model_name);

#### Extract Model Results

In [ ]:
# load the most recent model checkpoint
model, data, metadata, current_iter = kpms.load_checkpoint(project_dir, model_name)

# extract results
results = kpms.extract_results(model, metadata, project_dir, model_name)

#### Save Results as csv

In [ ]:
# optionally save results as csv
kpms.save_results_as_csv(results, project_dir, model_name)

### Apply to New Data

In [ ]:
# load the most recent model checkpoint and pca object
model = kpms.load_checkpoint(project_dir, model_name)[0]

# load new data (e.g. from deeplabcut)
new_data = 'path/to/new/data/' # can be a file, a directory, or a list of files
coordinates, confidences, bodyparts = kpms.load_keypoints(new_data, 'deeplabcut')
data, metadata = kpms.format_data(coordinates, confidences, **config())

# apply saved model to new data
results = kpms.apply_model(model, data, metadata, project_dir, model_name, **config())

# optionally rerun `save_results_as_csv` to export the new results
# kpms.save_results_as_csv(results, project_dir, model_name)

## Visualization

### Trajectory Plots

In [ ]:
results = kpms.load_results(project_dir, model_name)
kpms.generate_trajectory_plots(coordinates, results, project_dir, model_name, **config())

### Grid Movies

In [ ]:
kpms.generate_grid_movies(results, project_dir, model_name, coordinates=coordinates, **config());

### Syllable Dendrogram

In [ ]:
kpms.plot_similarity_dendrogram(coordinates, results, project_dir, model_name, **config())